#### Modelamiento para funcion de recomendación de juegos (ML)

In [95]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Carga de datos

In [96]:
df = pd.read_parquet('Datasets/df_merge.parquet')
df.head(3)

,genres_clean,app_name,price,item_id,developer,release_year,item_name,playtime_forever,user_id,items_count,recommend,review,posted_year,sentiment_analysis
0,Action,Half-Life,9.99,70,Valve,1998,Half-Life,1395.0,EizanAratoFujimaki,391.0,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,2015.0,2.0
1,Action,Half-Life,9.99,70,Valve,1998,Half-Life,590.0,GamerFag,64.0,True,If you own any half life but not this one that...,2011.0,0.0
2,Action,Half-Life,9.99,70,Valve,1998,Half-Life,5599.0,76561198020928326,110.0,True,this game has had me hooked for over 4 years a...,2014.0,2.0


In [97]:
#Creo un nuevo dataframe con las columnas necesarias para el modelo
df1 = df[['item_id','app_name','genres_clean']].copy()

In [98]:
df1.rename(columns={'genres_clean': 'genres'},inplace=True)
df1['item_id'] = df['item_id'].astype(int)

In [115]:
# Crea las variables dummi para la columna especificada
modelado = pd.get_dummies(df1, columns=['genres'], prefix="")
df_modelo = modelado.groupby(["item_id","app_name"]).sum().reset_index()
df_modelo.sample(3)

,item_id,app_name,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Documentary,_Free to Play,_Indie,_Massively Multiplayer,_Platformer,_RPG,_Racing,_Simulation,_Sports,_Strategy,_Utilities,_Video Production
695,214250,I Am Alive,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1647,318230,Convoy,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
369,42920,The Kings' Crusade,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [101]:
# Calcula la similitud de coseno para todas las filas desde la tercer hasta la ultima columna
similitudes = cosine_similarity(df_modelo.iloc[:, 2:])

In [102]:
# La funcion recibe un id de un juego y en base a este retorna una lista con juegos similares
def recomendacion_juego(id:int):
    
    # Crea un df con los datos de el id pasado como parametro
    juego = df_modelo[df_modelo['item_id'] == id]
    
    # En caso que el ID no este en el df, muestra el siguiente mensaje:
    if id not in juego.values:
        return f"El juego con ID {id} no se encuentra."

    # En caso de estar. Obtiene el índice del juego en el DataFrame.
    idx = juego.index
    idx = idx[0]

    # Tamaño de la muestra
    muestra = 2000

    # Ajustamos la semilla aleatoria
    df_sample = df_modelo.sample(n=muestra, random_state=42)
    
    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([df_modelo.iloc[idx, 3:]], df_sample.iloc[:, 3:])
    
    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"Juegos similares": similar_game_names}


In [110]:
recomendacion_juego(20)

{'Juegos similares': ['Tales of Zestiria',
  'Space Hulk Ascension',
  'Delta Force 2',
  'Tribes: Ascend',
  'Tropico 3']}